In [2]:
# The code was removed by Watson Studio for sharing.

# Coursera capstone project notebook

# Segmenting and Clustering Neighborhoods in Toronto

# Web scrapping the data 

In [3]:

import bs4 as bs
import urllib.request
import pandas as pd

source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'lxml')
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0


table = soup.find('table', attrs={'class':'wikitable sortable'})
table_rows = table.find_all('tr')

In [4]:
res=[]
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

### Creating the data frame

In [5]:
df=pd.DataFrame(res, columns=["PostalCode", "Borough","Neighbourhood"])
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Remove the not assigned borough

In [6]:
df = df[df.Borough != 'Not assigned']
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [7]:

df['Borough'].value_counts()



Etobicoke           45
North York          38
Scarborough         38
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

### Merge the duplicated Postcode and Borough Neighborhoods

In [8]:
df = df.groupby(['Borough','PostalCode'])['Neighbourhood'].apply(','.join).reset_index()
df.head()

,Borough,PostalCode,Neighbourhood
0,Central Toronto,M4N,Lawrence Park
1,Central Toronto,M4P,Davisville North
2,Central Toronto,M4R,North Toronto West
3,Central Toronto,M4S,Davisville
4,Central Toronto,M4T,"Moore Park,Summerhill East"


###  Check to see if there are 'Not assigned' neighbourhoods

In [9]:
df[df['Neighbourhood'].str.contains("Not assigned")]

,Borough,PostalCode,Neighbourhood
74,Queen's Park,M7A,Not assigned


### The neighbourhood that is not assigned is assigned the Borough name

In [10]:
 df['Neighbourhood'].replace(['Not assigned'],'Queens Park')

0                                          Lawrence Park
1                                       Davisville North
2                                     North Toronto West
3                                             Davisville
4                             Moore Park,Summerhill East
5      Deer Park,Forest Hill SE,Rathnelly,South Hill,...
6                                               Roselawn
7                     Forest Hill North,Forest Hill West
8                      The Annex,North Midtown,Yorkville
9                                               Rosedale
10                            Cabbagetown,St. James Town
11                                  Church and Wellesley
12                              Harbourfront,Regent Park
13                               Ryerson,Garden District
14                                        St. James Town
15                                           Berczy Park
16                                    Central Bay Street
17                             

### print the dimension of the dataframe

In [11]:
df.shape
#There are 103 rows

(103, 3)

### I tried to install the geocoder library for calling the google API but it took forever so i decided to download the data and loaded it to the IBM DB2 database and read it to a dataframe and called it df_data_3

In [13]:

#INSTALL THE LIBRARIES FOR THE NEXT SESSION 

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  10.41 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  30.61 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  31.34 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  38.68 MB/s
Libraries imported.


In [14]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### I merged the two dataframes and droped the extra Postal Code column 

In [15]:
df1=pd.merge(df,df_data_3, left_on='PostalCode', right_on='Postal Code', how='left').drop('Postal Code', axis=1)

In [16]:
df1.head()

,Borough,PostalCode,Neighbourhood,Latitude,Longitude
0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790
1,Central Toronto,M4P,Davisville North,43.712751,-79.390197
2,Central Toronto,M4R,North Toronto West,43.715383,-79.405678
3,Central Toronto,M4S,Davisville,43.704324,-79.388790
4,Central Toronto,M4T,"Moore Park,Summerhill East",43.689574,-79.383160


### Just checking the dimension of the new dataframe

In [17]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df1['Borough'].unique()),
        df1.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


### Use geopy library to get the latitude and longitude values of Toronto.
### In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.


In [18]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Create a map of Toronto with neighborhoods superimposed on top

In [19]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Simplify the above map and segment and cluster only the neighborhoods in Central Toronto. So let's slice the original dataframe and create a new dataframe of the central_toronto data.

In [20]:
central_toronto = df1[df1['Borough'] == 'Central Toronto'].reset_index(drop=True)
central_toronto.head()

,Borough,PostalCode,Neighbourhood,Latitude,Longitude
0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790
1,Central Toronto,M4P,Davisville North,43.712751,-79.390197
2,Central Toronto,M4R,North Toronto West,43.715383,-79.405678
3,Central Toronto,M4S,Davisville,43.704324,-79.388790
4,Central Toronto,M4T,"Moore Park,Summerhill East",43.689574,-79.383160


### Get the geographical location of Central toronto

In [21]:
address = 'Central Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.653963, -79.387207.


In [22]:
# create map of Central toronto using latitude and longitude values
map_central_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(central_toronto['Latitude'], central_toronto['Longitude'], central_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_central_toronto)  
    
map_central_toronto

### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


In [23]:

CLIENT_ID = '5RSJPVGGCRLVX5GY2JCDKYKW3K0AOBA1WZ5HO4EWXWDLFGQX' # your Foursquare ID
CLIENT_SECRET = 'SZR1WWGB1L3WWOKZX1HKH1ETZ4HHAS25YO3ROBW4IQXSREUM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


Your credentails:
CLIENT_ID: 5RSJPVGGCRLVX5GY2JCDKYKW3K0AOBA1WZ5HO4EWXWDLFGQX
CLIENT_SECRET:SZR1WWGB1L3WWOKZX1HKH1ETZ4HHAS25YO3ROBW4IQXSREUM


In [24]:
#Let's explore the first neighborhood in our dataframe.
#Get the neighborhood's name.

central_toronto.loc[0, 'Neighbourhood']

'Lawrence Park'

In [25]:
#Lets get the latitudes and longitudes of that neighbourhood
neighborhood_latitude = central_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = central_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = central_toronto.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


### Now, let's get the top 100 venues that are in Lawrence Park within a radius of 500 meters.
### First, let's create the GET request URL. Name your URL url.


### create the GET request URL. 

In [26]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=5RSJPVGGCRLVX5GY2JCDKYKW3K0AOBA1WZ5HO4EWXWDLFGQX&client_secret=SZR1WWGB1L3WWOKZX1HKH1ETZ4HHAS25YO3ROBW4IQXSREUM&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c8ff9639fb6b753bcb6030f'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-50e6da19e4b0d8a78a0e9794-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d163941735',
         'name': 'Park',
         'pluralName': 'Parks',
         'primary': True,
         'shortName': 'Park'}],
       'id': '50e6da19e4b0d8a78a0e9794',
       'location': {'address': '3055 Yonge Street',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Lawrence Avenue East',
        'distance': 465,
        'formattedAddress': ['3055 Yonge Street (Lawrence Avenue East)',
         'Toronto ON',
         'Canada'],
        'labeledLatLngs': [{

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
# Clean the json and structure it into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [30]:
#Check the number of venues returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


###  Explore Neighborhoods in central toronto
### Let's create a function to repeat the same process to all the neighborhoods in central toronto

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Run the above function on each neighborhood and create a new dataframe called central_toronto_venues

In [32]:

central_toronto_venues = getNearbyVenues(names=central_toronto['Neighbourhood'],
                                   latitudes=central_toronto['Latitude'],
                                   longitudes=central_toronto['Longitude']
                                  )

Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville


### Check the size of the new dataframe

In [34]:

print(central_toronto_venues.shape)
central_toronto_venues.head()

(109, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park
4,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop


### Let's check how many venues were returned for each neighborhood

In [35]:

central_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,32,32,32,32,32,32
Davisville North,8,8,8,8,8,8
"Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West",15,15,15,15,15,15
"Forest Hill North,Forest Hill West",4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park,Summerhill East",2,2,2,2,2,2
North Toronto West,20,20,20,20,20,20
Roselawn,1,1,1,1,1,1
"The Annex,North Midtown,Yorkville",24,24,24,24,24,24


### Let's find out how many unique categories can be curated from all the returned venues


In [36]:

print('There are {} uniques categories.'.format(len(central_toronto_venues['Venue Category'].unique())))

There are 58 uniques categories.


### Analyze Each Neighborhood


In [39]:

# one hot encoding
import pandas as pd
central_onehot = pd.get_dummies(central_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe

central_onehot['Neighborhood'] = central_toronto_venues['Neighborhood'] 


# move neighborhood column to the first column

fixed_columns = [central_onehot.columns[-1]] + list(central_onehot.columns[:-1])

central_onehot = central_onehot[fixed_columns]

central_onehot.head()


,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gourmet Shop,Greek Restaurant,Gym,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Medical Center,Mexican Restaurant,Miscellaneous Shop,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:

#And let's examine the new dataframe size
central_onehot.shape

(109, 59)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [41]:

central_grouped = central_onehot.groupby('Neighborhood').mean().reset_index()

central_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gourmet Shop,Greek Restaurant,Gym,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Medical Center,Mexican Restaurant,Miscellaneous Shop,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.000000,0.000,0.03125,0.031250,0.000000,0.0625,0.000000,0.00,0.000,0.062500,0.000000,0.000000,0.09375,0.03125,0.03125,0.00,0.000,0.000000,0.0,0.03125,0.03125,0.03125,0.000000,0.000,0.031250,0.0625,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.031250,0.031250,0.093750,0.0,0.000000,0.00,0.0625,0.00,0.09375,0.03125,0.00,0.0,0.000000,0.000000,0.062500,0.000000,0.0,0.03125,0.03125,0.00,0.000000,0.000000,0.00
1,Davisville North,0.000000,0.000000,0.000000,0.125,0.00000,0.125000,0.000000,0.0000,0.000000,0.00,0.125,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00,0.125,0.000000,0.0,0.00000,0.00000,0.12500,0.000000,0.125,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.125000,0.000000,0.000000,0.0,0.000000,0.00,0.0000,0.00,0.12500,0.00000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.00000,0.00,0.000000,0.000000,0.00
2,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0.066667,0.000000,0.066667,0.000,0.00000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.133333,0.066667,0.000000,0.00000,0.00000,0.00000,0.00,0.000,0.066667,0.0,0.00000,0.00000,0.00000,0.000000,0.000,0.000000,0.0000,0.00,0.000000,0.066667,0.000000,0.066667,0.00,0.00,0.000000,0.000000,0.066667,0.0,0.133333,0.00,0.0000,0.00,0.00000,0.00000,0.00,0.0,0.066667,0.066667,0.066667,0.000000,0.0,0.00000,0.00000,0.00,0.000000,0.066667,0.00
3,"Forest Hill North,Forest Hill West",0.000000,0.000000,0.000000,0.000,0.00000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00,0.000,0.000000,0.0,0.00000,0.00000,0.00000,0.000000,0.000,0.000000,0.0000,0.25,0.000000,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0000,0.00,0.00000,0.00000,0.00,0.0,0.000000,0.000000,0.250000,0.000000,0.0,0.00000,0.00000,0.25,0.000000,0.000000,0.00
4,Lawrence Park,0.000000,0.000000,0.000000,0.000,0.00000,0.000000,0.333333,0.0000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00,0.000,0.000000,0.0,0.00000,0.00000,0.00000,0.000000,0.000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.333333,0.000000,0.000000,0.0,0.000000,0.00,0.0000,0.00,0.00000,0.00000,0.00,0.0,0.000000,0.000000,0.000000,0.333333,0.0,0.00000,0.00000,0.00,0.000000,0.000000,0.00
5,"Moore Park,Summerhill East",0.000000,0.000000,0.000000,0.000,0.00000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00,0.000,0.000000,0.0,0.00000,0.00000,0.00000,0.000000,0.000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.5,0.000000,0.00,0.0000,0.00,0.00000,0.00000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.5,0.00000,0.00000,0.00,0.000000,0.000000,0.00
6,North Toronto West,0.000000,0.000000,0.000000,0.000,0.00000,0.000000,0.000000,0.0000,0.000000,0.05,0.150,0.100000,0.000000,0.050000,0.05000,0.05000,0.00000,0.05,0.000,0.000000,0.0,0.00000,0.00000,0.00000,0.000000,0.000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.05,0.05,0.050000,0.000000,0.000000,0.0,0.000000,0.05,0.0000,0.05,0.05000,0.00000,0.05,0.1,0.000000,0.000000,0.000000,0.000000,0.

In [42]:
#check the shape of the new dataframe
central_grouped.shape

(9, 59)

### Let's print each neighborhood along with the top 5 most common venues


In [43]:

num_top_venues = 5

for hood in central_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = central_grouped[central_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
            venue  freq
0     Pizza Place  0.09
1  Sandwich Place  0.09
2    Dessert Shop  0.09
3      Restaurant  0.06
4            Café  0.06


----Davisville North----
            venue  freq
0  Sandwich Place  0.12
1             Gym  0.12
2  Breakfast Spot  0.12
3            Park  0.12
4    Burger Joint  0.12


----Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West----
                 venue  freq
0                  Pub  0.13
1          Coffee Shop  0.13
2  American Restaurant  0.07
3     Sushi Restaurant  0.07
4          Pizza Place  0.07


----Forest Hill North,Forest Hill West----
                venue  freq
0       Jewelry Store  0.25
1               Trail  0.25
2  Mexican Restaurant  0.25
3    Sushi Restaurant  0.25
4  Salon / Barbershop  0.00


----Lawrence Park----
                 venue  freq
0                 Park  0.33
1             Bus Line  0.33
2          Swim School  0.33
3  American Restaurant  0.00
4       Sandwich Place  0.00


----Moo

In [44]:
#Sort the venues in descending order and put that into a dataframe

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = central_grouped['Neighborhood']

for ind in np.arange(central_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(central_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Pizza Place,Sandwich Place,Coffee Shop,Sushi Restaurant,Restaurant,Italian Restaurant,Café,Seafood Restaurant,Farmers Market
1,Davisville North,Clothing Store,Hotel,Breakfast Spot,Gym,Burger Joint,Sandwich Place,Food & Drink Shop,Park,Yoga Studio,Fast Food Restaurant
2,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Pub,Coffee Shop,American Restaurant,Sports Bar,Vietnamese Restaurant,Light Rail Station,Fried Chicken Joint,Medical Center,Pizza Place,Convenience Store
3,"Forest Hill North,Forest Hill West",Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Cosmetics Shop,History Museum,Gym,Greek Restaurant,Gourmet Shop,Garden
4,Lawrence Park,Swim School,Bus Line,Park,Yoga Studio,Dessert Shop,Hotel,History Museum,Gym,Greek Restaurant,Gourmet Shop


### Cluster Neighborhoods


In [46]:

#Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5
central_grouped_clustering = central_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(central_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 4, 3, 2, 0, 1, 0], dtype=int32)

In [49]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
#neighborhoods_venues_sorted.insert(0,'Cluster Labels',kmeans.labels_)

central_merged = central_toronto_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood

central_merged = central_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
central_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park,3,Swim School,Bus Line,Park,Yoga Studio,Dessert Shop,Hotel,History Museum,Gym,Greek Restaurant,Gourmet Shop
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School,3,Swim School,Bus Line,Park,Yoga Studio,Dessert Shop,Hotel,History Museum,Gym,Greek Restaurant,Gourmet Shop
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line,3,Swim School,Bus Line,Park,Yoga Studio,Dessert Shop,Hotel,History Museum,Gym,Greek Restaurant,Gourmet Shop
3,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park,0,Clothing Store,Hotel,Breakfast Spot,Gym,Burger Joint,Sandwich Place,Food & Drink Shop,Park,Yoga Studio,Fast Food Restaurant
4,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop,0,Clothing Store,Hotel,Breakfast Spot,Gym,Burger Joint,Sandwich Place,Food & Drink Shop,Park,Yoga Studio,Fast Food Restaurant


### Visualizing the resulting clusters

In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(central_merged['Neighborhood Latitude'], central_merged['Neighborhood Longitude'], central_merged['Neighborhood'], central_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters
### Examining each cluster and determining the discriminating venue categories that distinguish each cluster.

### cluster 1: Hotel and clothing store

In [52]:
central_merged.loc[central_merged['Cluster Labels'] == 0, central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,43.712751,-79.387776,Park,0,Clothing Store,Hotel,Breakfast Spot,Gym,Burger Joint,Sandwich Place,Food & Drink Shop,Park,Yoga Studio,Fast Food Restaurant
4,43.712751,-79.392881,Food & Drink Shop,0,Clothing Store,Hotel,Breakfast Spot,Gym,Burger Joint,Sandwich Place,Food & Drink Shop,Park,Yoga Studio,Fast Food Restaurant
5,43.712751,-79.391557,Breakfast Spot,0,Clothing Store,Hotel,Breakfast Spot,Gym,Burger Joint,Sandwich Place,Food & Drink Shop,Park,Yoga Studio,Fast Food Restaurant
6,43.712751,-79.390880,Hotel,0,Clothing Store,Hotel,Breakfast Spot,Gym,Burger Joint,Sandwich Place,Food & Drink Shop,Park,Yoga Studio,Fast Food Restaurant
7,43.712751,-79.390473,Sandwich Place,0,Clothing Store,Hotel,Breakfast Spot,Gym,Burger Joint,Sandwich Place,Food & Drink Shop,Park,Yoga Studio,Fast Food Restaurant
8,43.712751,-79.390296,Burger Joint,0,Clothing Store,Hotel,Breakfast Spot,Gym,Burger Joint,Sandwich Place,Food & Drink Shop,Park,Yoga Studio,Fast Food Restaurant
9,43.712751,-79.393537,Gym,0,Clothing Store,Hotel,Breakfast Spot,Gym,Burger Joint,Sandwich Place,Food & Drink Shop,Park,Yoga Studio,Fast Food Restaurant
10,43.712751,-79.393873,Clothing Store,0,Clothing Store,Hotel,Breakfast Spot,Gym,Burger Joint,Sandwich Place,Food & Drink Shop,Park,Yoga Studio,Fast Food Restaurant
11,43.715383,-79.400091,Diner,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Fast Food Restaurant,Mexican Restaurant,Miscellaneous Shop,Park,Diner,Dessert Shop,Cosmetics Shop
12,43.715383,-79.400109,Yoga Studio,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Fast Food Restaurant,Mexican Restaurant,Miscellaneous Shop,Park,Diner,Dessert Shop,Cosmetics Shop


### cluster 2: Garden venues

In [53]:
central_merged.loc[central_merged['Cluster Labels'] == 1, central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
80,43.711695,-79.411978,Garden,1,Garden,Yoga Studio,Dessert Shop,Indian Restaurant,Hotel,History Museum,Gym,Greek Restaurant,Gourmet Shop,Fried Chicken Joint


### cluster 3

In [54]:
central_merged.loc[central_merged['Cluster Labels'] == 2, central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,43.689574,-79.382900,Tennis Court,2,Tennis Court,Playground,Yoga Studio,Dessert Shop,Hotel,History Museum,Gym,Greek Restaurant,Gourmet Shop,Garden
64,43.689574,-79.383465,Playground,2,Tennis Court,Playground,Yoga Studio,Dessert Shop,Hotel,History Museum,Gym,Greek Restaurant,Gourmet Shop,Garden


### cluster 4: Recreational venues

In [55]:
central_merged.loc[central_merged['Cluster Labels'] == 3, central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.72802,-79.394382,Park,3,Swim School,Bus Line,Park,Yoga Studio,Dessert Shop,Hotel,History Museum,Gym,Greek Restaurant,Gourmet Shop
1,43.72802,-79.382860,Swim School,3,Swim School,Bus Line,Park,Yoga Studio,Dessert Shop,Hotel,History Museum,Gym,Greek Restaurant,Gourmet Shop
2,43.72802,-79.382805,Bus Line,3,Swim School,Bus Line,Park,Yoga Studio,Dessert Shop,Hotel,History Museum,Gym,Greek Restaurant,Gourmet Shop


### cluster 5 :Beauty venues

In [56]:
central_merged.loc[central_merged['Cluster Labels'] == 4, central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
81,43.696948,-79.410101,Trail,4,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Cosmetics Shop,History Museum,Gym,Greek Restaurant,Gourmet Shop,Garden
82,43.696948,-79.407957,Sushi Restaurant,4,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Cosmetics Shop,History Museum,Gym,Greek Restaurant,Gourmet Shop,Garden
83,43.696948,-79.407644,Jewelry Store,4,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Cosmetics Shop,History Museum,Gym,Greek Restaurant,Gourmet Shop,Garden
84,43.696948,-79.407605,Mexican Restaurant,4,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Cosmetics Shop,History Museum,Gym,Greek Restaurant,Gourmet Shop,Garden
